**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [2]:
!pip install scikit-video

     |████████████████████████████████| 2.3MB 2.8MB/s 


In [3]:
import random
import keras
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from tensorflow.python.util import deprecation
deprecation._PRINT_DEPRECATION_WARNINGS = False
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense, Flatten
from keras.layers import Input
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization

Using TensorFlow backend.


Instructions for updating:
non-resource variables are not supported in the long term


# MiniProject on Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [0]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [0]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

The function *act* returns the next action to take based on an $\epsilon$-greedy policy: it chooses between exploration (random action) and exploitation (best known action). The goal is to explore new policies to find a better one, while not "losing" too much in the process by exploiting the good policies we already know. The parameter $\epsilon$ regulates the balance between the exploration and exploitation: the higher it is the more important the exploration.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [0]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0


        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [0]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=20 
epochs_test=10

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

The *position* matrix encodes the current position of the agent (1), the cells it can go too (0) and the borders (-1).


The *board* matrix encodes the reward of the cells on the board: +0.5 if it is cheeses, -1 for position or 0 for ordinary cells.



## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [0]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        # The agent chooses a random action each time
        return np.random.randint(0, self.n_action)

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [0]:
def test(agent, env, epochs, prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
        ### ------------
        # Initialization

        # Reset the environment and game
        state = env.reset()
        
        # Initialization counters of points
        win = 0
        lose = 0


        ### -------------------------
        # Play until the game is over
        over = False
        while not over:

            # Take action
            action = agent.act(state)
            state, reward, over = env.act(action)

            # Count lost points and rewards
            if reward > 0:
                  win = win + reward
            if reward < 0:
                  lose = lose - reward

            # Update frame to display
            env.get_frame(e)


        ### ----------
        # End the game

        # Save as a mp4
        env.draw(prefix + str(e))

        # Update stats
        score = score + win - lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [0]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 8.5/15.0. Average score (-6.5)
Win/lose count 10.5/17.0. Average score (-6.5)
Win/lose count 12.5/11.0. Average score (-3.8333333333333335)
Win/lose count 10.5/13.0. Average score (-3.5)
Win/lose count 13.5/10.0. Average score (-2.1)
Win/lose count 8.5/14.0. Average score (-2.6666666666666665)
Win/lose count 11.0/9.0. Average score (-2.0)
Win/lose count 13.5/18.0. Average score (-2.3125)
Win/lose count 9.0/17.0. Average score (-2.9444444444444446)
Win/lose count 15.0/11.0. Average score (-2.25)
Final score: -2.25


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




**1.**
Let $\pi$ be the policy, and $\rho^\pi (s',a')$ the probability to be in state $(s',a')$.

\begin{align} 
Q^\pi(s,a) &= \mathbf{E}_{\rho^\pi} [\sum_{t  \ge 0} \gamma^{t}r(s_t,a_t)|s_0=s,a_0=a] \\ 
&= r(s, a) + \gamma \mathbf{E}_{\rho^\pi}[\sum_{t \ge 1} \gamma^{t - 1} r(s_t,a_t)|s_0=s,a_0=a] \\ 
&= r(s, a) + \gamma \sum_{s'} p(s_1 = s' | s_0 = s) \mathbf{E}_{\rho^\pi} [\sum_{t \ge 1} \gamma^{t - 1} r(s_t,a_t)|s_1=s',a_1=a'] \\ 
&= r(s, a) + \gamma \sum_{s'} p(s_1 = s' | s_0 = s) Q^\pi(s', a') \\ 
\end{align} 

Finally,
\begin{align}
Q^\pi(s,a) = \mathbf{E}_{(s',a')\sim p(.|s,a)} [r(s,a)+\gamma Q^{\pi}(s',a')] 
\end{align} 

\\
**2.**
Let $\pi^{*}$ be the optimal policy.

By definition, we have :
\begin{align} 
Q^*(s, a) &= \max_{\pi} Q^{\pi}(s, a) 
\end{align} 

Hence,

\begin{align} 
Q^*(s, a) &= \max_{\pi} [ r(s,a) + \gamma \sum_{s'} p(s' | s,a) Q^{\pi}(s', a'=\pi(s')) ] \\ 
&= r(s,a) + \gamma \sum_{s'} p(s' | s,a) \max_{\pi} Q^{\pi}(s', \pi(s')) \\ 
&\leq r(s,a) + \gamma \sum_{s'} p(s' | s,a) \max_{\pi, a'} Q^{\pi}(s', a') \quad \mathbf{(*)}\\ 
& = r(s,a) + \gamma \sum_{s'} p(s' | s,a) \max_{ a'} Q^*(s', a') \\ 
&= \mathbf{E}_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^*(s',a')] 
\end{align} 


However we also have:
\begin{align}
\sum_{s'} p(s'|s, a) \max_{\pi, a'}  Q^\pi(s', a') 
&=  \sum_{s'} p(s'|s, a) \max_{a'} Q^*(s', a') \\
&\leq \max_{\pi} \sum_{s'} p(s'|s, a) \max_{a'} Q^{\pi}(s', a')
\end{align}

Which gives the equality for line $(*)$ of the previous computation.

\\
**3.**
The previous results shows that $Q^*$ is a fixed point of the Optimal Bellman operator. Therefore, we aim to estimate $Q^*$ by finding at each step the $\pi$ minimizing the Mean Squared Error :


\begin{align} 
& \mathbf{E}_{s'\sim \pi^*}(.|s,a)\left[r(s,a)+\gamma\max_{a'}Q^{\pi}(s',a')\right] - Q^{\pi}(s,a) 
\end{align} 


***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [0]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        # Remove the first element if the memory is full
        if len(self.memory) == self.max_memory: 
            self.memory = self.memory[1:]

        # Store the new value in the memory list
        self.memory.append(m)
        

    def random_access(self):
        # Returns a random element from the memory list
        return random.choice(self.memory)

***
The pipeline we will use for training is given below:

In [0]:
def train(agent,env,epoch,prefix='',verbose=True):
    # Number of won games
    score = 0
    loss = 0

    # Changed so that it starts at 1
    for e in range(1,epoch+1):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        
        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        if verbose or (e==epoch):
            print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
                  .format(e, epoch, loss, win, lose, win-lose))
            
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

    print('Final score: '+str(score/epoch))

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [0]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size


    def learned_act(self, s):
        # Returns the action with the highest score 
        # given the policy we have learned up until now
        return np.argmax(self.model.predict(np.array([s]))[0])


    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        ### Initialization
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            # Access a move that was stored in the memory, chosen at random
            # we go at state s_
            s_, n_s_, a_, r_, game_over_ = self.memory.random_access()
            input_states[i] = s_

            # Initialization of Q-function at the current state
            target_q[i] = self.model.predict(np.array([s_]))[0]  


            ### --------------------------
            # Update the target Q function

            # Adding discounted prediction if episode not finished
            # only reward 
            if game_over_:
                target_q[i, a_] = r_
            else:
                target_q[i, a_] = r_ 
                target_q[i, a_] += self.discount * self.model.predict(n_s_.reshape(1, 5, 5, self.n_state)).max()


        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)

        return l


    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            

    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        model = Sequential()
        model.add(Flatten(input_shape=(5,5,2)))
        model.add(Dense(128))
        model.add(Activation('relu'))
        model.add(Dense(64))
        model.add(Activation("relu"))
        model.add(Dense(4))
        model.add(Activation('linear'))

        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        

#### Testing different parameters

Has been launched several times with different ranges of parameters

In [0]:
for learning_rate in [1,0.5]:
    for batch in [64,128]:
        for memory in [2000,4000]:
            print("\nLR=",learning_rate,"; batch size=",batch,"; memory size=",memory)
            env = Environment(grid_size=size, max_time=T, temperature=0.3)
            agent = DQN_FC(size, lr=learning_rate, epsilon = 0.1, memory_size=memory, batch_size = batch)
            train(agent, env, epochs_train, prefix='fc_train', verbose=False)
            HTML(display_videos('fc_train10.mp4'))


LR= 1 ; batch size= 64 ; memory size= 2000
Epoch 020/020 | Loss nan | Win/lose count 1.5/0 (1.5)
Final score: -0.5

LR= 1 ; batch size= 64 ; memory size= 4000
Epoch 020/020 | Loss nan | Win/lose count 1.5/3.0 (-1.5)
Final score: -1.025

LR= 1 ; batch size= 128 ; memory size= 2000
Epoch 020/020 | Loss nan | Win/lose count 1.5/3.0 (-1.5)
Final score: -1.9

LR= 1 ; batch size= 128 ; memory size= 4000
Epoch 020/020 | Loss nan | Win/lose count 2.0/4.0 (-2.0)
Final score: -0.425

LR= 0.5 ; batch size= 64 ; memory size= 2000
Epoch 020/020 | Loss nan | Win/lose count 2.5/1.0 (1.5)
Final score: -1.35

LR= 0.5 ; batch size= 64 ; memory size= 4000
Epoch 020/020 | Loss nan | Win/lose count 3.0/0 (3.0)
Final score: -0.5

LR= 0.5 ; batch size= 128 ; memory size= 2000
Epoch 020/020 | Loss 0.0015 | Win/lose count 10.5/2.0 (8.5)
Final score: 2.05

LR= 0.5 ; batch size= 128 ; memory size= 4000
Epoch 020/020 | Loss nan | Win/lose count 3.5/2.0 (1.5)
Final score: -1.175


#### Relaunching with best parameters

In [0]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=1, epsilon = 0.1, memory_size=4000, batch_size = 128)
train(agent, env, epochs_train, prefix='fc_train', verbose=True)
HTML(display_videos('fc_train10.mp4'))

Epoch 001/020 | Loss 0.0048 | Win/lose count 2.5/6.0 (-3.5)
Epoch 002/020 | Loss 0.0021 | Win/lose count 3.5/5.0 (-1.5)
Epoch 003/020 | Loss 0.0148 | Win/lose count 3.5/5.0 (-1.5)
Epoch 004/020 | Loss 0.0027 | Win/lose count 5.0/2.0 (3.0)
Epoch 005/020 | Loss 0.0113 | Win/lose count 4.0/4.0 (0.0)
Epoch 006/020 | Loss 0.0029 | Win/lose count 7.0/1.0 (6.0)
Epoch 007/020 | Loss 0.0116 | Win/lose count 7.5/2.0 (5.5)
Epoch 008/020 | Loss 0.0281 | Win/lose count 7.0/4.0 (3.0)
Epoch 009/020 | Loss 0.0049 | Win/lose count 4.5/3.0 (1.5)
Epoch 010/020 | Loss 0.0021 | Win/lose count 10.5/6.0 (4.5)
Epoch 011/020 | Loss 0.0142 | Win/lose count 4.5/4.0 (0.5)
Epoch 012/020 | Loss 0.0094 | Win/lose count 8.5/2.0 (6.5)
Epoch 013/020 | Loss 0.0014 | Win/lose count 12.0/4.0 (8.0)
Epoch 014/020 | Loss 0.0111 | Win/lose count 7.0/5.0 (2.0)
Epoch 015/020 | Loss 0.0023 | Win/lose count 11.5/4.0 (7.5)
Epoch 016/020 | Loss 0.0018 | Win/lose count 10.0/4.0 (6.0)
Epoch 017/020 | Loss 0.0176 | Win/lose count 6.0/

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [0]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        # Model with 2 Convolutional layers and a final fully connected one
        model = Sequential()
        model.add(Conv2D(filters=32,kernel_size=3,input_shape=(5,5,self.n_state)))
        model.add(Activation('relu'))
        model.add(Conv2D(filters=16,kernel_size=3))
        model.add(Activation('relu'))
        model.add(Flatten())
        model.add(Dense(4))
        model.add(Activation('linear'))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [0]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=0.1, epsilon = 0.1, memory_size=4000, batch_size = 128)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train10.mp4'))

Epoch 001/020 | Loss 0.0048 | Win/lose count 3.0/4.0 (-1.0)
Epoch 002/020 | Loss 0.0040 | Win/lose count 3.0/4.0 (-1.0)
Epoch 003/020 | Loss 0.0043 | Win/lose count 5.5/5.0 (0.5)
Epoch 004/020 | Loss 0.0050 | Win/lose count 1.0/1.0 (0.0)
Epoch 005/020 | Loss 0.0070 | Win/lose count 6.0/5.0 (1.0)
Epoch 006/020 | Loss 0.0030 | Win/lose count 4.0/7.0 (-3.0)
Epoch 007/020 | Loss 0.0045 | Win/lose count 4.5/3.0 (1.5)
Epoch 008/020 | Loss 0.0131 | Win/lose count 5.5/3.0 (2.5)
Epoch 009/020 | Loss 0.0046 | Win/lose count 5.5/5.0 (0.5)
Epoch 010/020 | Loss 0.0086 | Win/lose count 5.0/5.0 (0.0)
Epoch 011/020 | Loss 0.0061 | Win/lose count 4.0/5.0 (-1.0)
Epoch 012/020 | Loss 0.0082 | Win/lose count 3.5/5.0 (-1.5)
Epoch 013/020 | Loss 0.0053 | Win/lose count 3.0/3.0 (0.0)
Epoch 014/020 | Loss 0.0291 | Win/lose count 3.0/1.0 (2.0)
Epoch 015/020 | Loss 0.0169 | Win/lose count 5.5/5.0 (0.5)
Epoch 016/020 | Loss 0.0040 | Win/lose count 4.0/2.0 (2.0)
Epoch 017/020 | Loss 0.0154 | Win/lose count 5.5/3.

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [0]:
### Testing different temperatures

for temp in [0.1,0.3,0.5,0.7,0.9]:
    env = Environment(grid_size=size, max_time=T,temperature=temp)

    agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=4000, batch_size = 128)
    agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

    agent_fc = DQN_FC(size, lr=1, epsilon = 0.1, memory_size=4000, batch_size = 128)
    agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')


    ### ------------------
    # Testing the 2 models
    print('Temperature:',temp)
    print('\nTest of the CNN')
    test(agent_cnn,env,epochs_test,prefix='cnn_test')


    print('\nTest of the FC')
    test(agent_fc,env,epochs_test,prefix='fc_test')

Temperature: 0.1

Test of the CNN
Win/lose count 0.5/1.0. Average score (-0.5)
Win/lose count 0.5/2.0. Average score (-1.0)
Win/lose count 2.0/0. Average score (0.0)
Win/lose count 2.0/0. Average score (0.5)
Win/lose count 2.0/3.0. Average score (0.2)
Win/lose count 6.0/0. Average score (1.1666666666666667)
Win/lose count 2.5/0. Average score (1.3571428571428572)
Win/lose count 1.5/1.0. Average score (1.25)
Win/lose count 1.5/0. Average score (1.2777777777777777)
Win/lose count 2.0/2.0. Average score (1.15)
Final score: 1.15

Test of the FC
Win/lose count 1.0/1.0. Average score (0.0)
Win/lose count 1.0/0. Average score (0.5)
Win/lose count 1.0/2.0. Average score (0.0)
Win/lose count 1.5/0. Average score (0.375)
Win/lose count 0.5/1.0. Average score (0.2)
Win/lose count 2.0/0. Average score (0.5)
Win/lose count 0.5/0. Average score (0.5)
Win/lose count 0.5/1.0. Average score (0.375)
Win/lose count 0.5/0. Average score (0.3888888888888889)
Win/lose count 0/0. Average score (0.35)
Final s

In [0]:
HTML(display_videos('cnn_test9.mp4'))

In [0]:
HTML(display_videos('fc_test9.mp4'))

**Answer**

Generally, a high temperature drives better results since there are much more bonuses available.

The main issue is that the agent does not explore enough especially with a low temperature. In that case, when the agent cannot see bonuses (more than 2 cells away), it will stay in the same vicinity.


***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [0]:
def train_explore(agent, env, epoch, prefix=''):
    # Number of won games
    score = 0
    loss = 0

    # Init steps
    step = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False
        win = 0
        lose = 0

        # Decreasing epsilon
        agent.set_epsilon(agent.epsilon * 0.9) 

        while not game_over:
            # The agent performs an action      
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action,train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state, action, reward, game_over)

        # Save as a mp4
        if (e + 1) % 10 == 0:
            env.draw(prefix + str(e + 1))

        # Update stats
        score += win-lose

        eps = agent.epsilon
        print("Epoch {:03d}/{:03d} | eps={:.2f} | Loss {:.4f} | Win/lose count {:.1f}/{:.1f} ({:.1f})"
              .format(e, epoch, eps, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

In [0]:
class EnvironmentExploring(Environment):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        # Inherits from the original Environment class
        Environment.__init__(self, grid_size=grid_size, max_time=max_time, temperature=temperature)

        self.malus_position = np.zeros((self.grid_size, self.grid_size))
        self.malus_position[self.x, self.y] = 0.1

        # To take into account the additionnal info in the state for the model
        self.n_state = 3
  
  
    def reset(self):
        # Reset environement
        state = Environment.reset(self)

        # Reset the position of the poisonous cells
        self.malus_position = np.zeros((self.grid_size, self.grid_size))
        self.malus_position[self.x, self.y] = 0.1

        # 3 "feature" states instead of 2
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        # We restrict the visibility of the agent to 2 cells
        state = state[self.x-2:self.x+3, self.y-2:self.y+3, :]

        return state


    def act(self, action, train=False):
        # Take an action like in the original Environment
        state, reward, game_over = Environment.act(self, action)

        # Updating the reward
        if train:
            # Penalization when the agent goes back to a cell it has already visited
            reward -= self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] = 0.1


        # 3 "feature" states instead of 2
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        # We restrict the visibility of the agent to 2 cells
        state = state[self.x-2:self.x+3, self.y-2:self.y+3, :]
        
        return state, reward, game_over
    

In [18]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.5)
agent = DQN_CNN(size, lr=.1, epsilon = 0.95, memory_size=4000, batch_size = 128, n_state=3)
train_explore(agent, env, 50, prefix='cnn_train_explore')

HTML(display_videos('cnn_train_explore10.mp4'))

Epoch 000/050 | eps=0.85 | Loss 0.0364 | Win/lose count 11.5/32.7 (-21.2)
Epoch 001/050 | eps=0.77 | Loss 0.0075 | Win/lose count 19.0/21.5 (-2.5)
Epoch 002/050 | eps=0.69 | Loss 0.0057 | Win/lose count 21.5/23.2 (-1.7)
Epoch 003/050 | eps=0.62 | Loss 0.0054 | Win/lose count 22.5/26.1 (-3.6)
Epoch 004/050 | eps=0.56 | Loss 0.0177 | Win/lose count 24.0/25.4 (-1.4)
Epoch 005/050 | eps=0.50 | Loss 0.0046 | Win/lose count 11.5/22.2 (-10.7)
Epoch 006/050 | eps=0.45 | Loss 0.0077 | Win/lose count 29.5/23.7 (5.8)
Epoch 007/050 | eps=0.41 | Loss 0.0179 | Win/lose count 23.5/20.7 (2.8)
Epoch 008/050 | eps=0.37 | Loss 0.0305 | Win/lose count 28.0/16.9 (11.1)
Epoch 009/050 | eps=0.33 | Loss 0.0075 | Win/lose count 34.0/21.9 (12.1)
Epoch 010/050 | eps=0.30 | Loss 0.0067 | Win/lose count 33.0/16.8 (16.2)
Epoch 011/050 | eps=0.27 | Loss 0.0163 | Win/lose count 23.5/21.2 (2.3)
Epoch 012/050 | eps=0.24 | Loss 0.0049 | Win/lose count 29.0/15.8 (13.2)
Epoch 013/050 | eps=0.22 | Loss 0.0071 | Win/lose co

In [20]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore9.mp4'))

Win/lose count 10.0/0. Average score (10.0)
Win/lose count 30.0/0. Average score (20.0)
Win/lose count 23.5/0. Average score (21.166666666666668)
Win/lose count 33.5/1.0. Average score (24.0)
Win/lose count 39.0/4.0. Average score (26.2)
Win/lose count 22.5/0. Average score (25.583333333333332)
Win/lose count 22.5/1.0. Average score (25.0)
Win/lose count 36.0/3.0. Average score (26.0)
Win/lose count 24.5/0. Average score (25.833333333333332)
Win/lose count 27.0/2.0. Average score (25.75)
Final score: 25.75


***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***